In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from collections import deque, defaultdict
from scipy.optimize import linear_sum_assignment
import cv2
import os
import time
from pathlib import Path
from ultralytics import YOLO
import torchvision.transforms as transforms
from sklearn.cluster import DBSCAN
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

class EnhancedSportsReIDNet(nn.Module):
    """Enhanced Sports-optimized ReID network with attention and multi-scale features"""
    def __init__(self, feature_dim=768, num_classes=1000):
        super(EnhancedSportsReIDNet, self).__init__()

        # Enhanced backbone with residual connections
        self.backbone = self._build_backbone()

        # Multi-scale feature extraction with attention
        self.global_pool = nn.AdaptiveAvgPool2d(1)
        self.part_pool = nn.AdaptiveMaxPool2d((6, 1))  # 6 horizontal strips for body parts

        # Channel attention for feature refinement
        self.channel_attention = ChannelAttention(512)

        # Feature dimensions for different scales
        self.global_dim = 512
        self.part_dim = 512 * 6
        self.color_dim = 64

        # Feature projectors
        self.global_projector = nn.Sequential(
            nn.Linear(self.global_dim, feature_dim // 3),
            nn.BatchNorm1d(feature_dim // 3),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2)
        )

        self.part_projector = nn.Sequential(
            nn.Linear(self.part_dim, feature_dim // 3),
            nn.BatchNorm1d(feature_dim // 3),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2)
        )

        self.color_projector = nn.Sequential(
            nn.Linear(self.color_dim, feature_dim // 3),
            nn.BatchNorm1d(feature_dim // 3),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2)
        )

        # Final feature fusion with residual connection
        self.fusion_layer = nn.Sequential(
            nn.Linear(feature_dim, feature_dim),
            nn.BatchNorm1d(feature_dim),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(feature_dim, feature_dim)
        )

        # Classification head (for training if needed)
        self.classifier = nn.Linear(feature_dim, num_classes)

        # Initialize weights
        self._initialize_weights()

    def _build_backbone(self):
        """Build enhanced CNN backbone"""
        layers = []

        # First block
        layers.extend([
            nn.Conv2d(3, 64, 7, 2, 3, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(3, 2, 1)
        ])

        # Residual blocks with increasing channels
        in_channels = 64
        for i, (out_channels, num_blocks) in enumerate([(64, 2), (128, 2), (256, 2), (512, 2)]):
            for j in range(num_blocks):
                layers.append(ResidualBlock(
                    in_channels if j == 0 else out_channels,
                    out_channels,
                    stride=2 if (i > 0 and j == 0) else 1
                ))
            in_channels = out_channels

        return nn.Sequential(*layers)

    def _initialize_weights(self):
        """Initialize network weights"""
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

    def forward(self, x, return_features=True):
        """Forward pass with multi-scale feature extraction"""
        batch_size = x.size(0)

        # Backbone feature extraction
        features = self.backbone(x)  # [B, 512, H, W]

        # Apply channel attention
        features = self.channel_attention(features)

        # Global features
        global_feat = self.global_pool(features).view(batch_size, -1)  # [B, 512]
        global_feat = self.global_projector(global_feat)

        # Part-based features (horizontal strips)
        part_feat = self.part_pool(features).view(batch_size, -1)  # [B, 512*6]
        part_feat = self.part_projector(part_feat)

        # Color features from input (simple but effective)
        color_feat = self._extract_color_features(x)  # [B, 64]
        color_feat = self.color_projector(color_feat)

        # Combine all features
        combined_feat = torch.cat([global_feat, part_feat, color_feat], dim=1)

        # Final fusion with residual connection
        fused_feat = self.fusion_layer(combined_feat)
        final_feat = combined_feat + fused_feat  # Residual connection

        # L2 normalize for cosine similarity
        normalized_feat = F.normalize(final_feat, p=2, dim=1)

        if return_features:
            return normalized_feat
        else:
            # For training with classification loss
            logits = self.classifier(normalized_feat)
            return normalized_feat, logits

    def _extract_color_features(self, x):
        """Extract color features from input tensor"""
        batch_size = x.size(0)

        # Convert to HSV-like representation
        # Simple approximation of color statistics
        mean_rgb = torch.mean(x, dim=[2, 3])  # [B, 3]
        std_rgb = torch.std(x, dim=[2, 3])   # [B, 3]

        # Histogram-like features (simplified)
        # Split into regions and compute statistics
        h, w = x.size(2), x.size(3)

        # Upper body region (jersey colors)
        upper_region = x[:, :, :h//2, :]
        upper_mean = torch.mean(upper_region, dim=[2, 3])
        upper_std = torch.std(upper_region, dim=[2, 3])

        # Lower body region
        lower_region = x[:, :, h//2:, :]
        lower_mean = torch.mean(lower_region, dim=[2, 3])
        lower_std = torch.std(lower_region, dim=[2, 3])

        # Combine color statistics
        color_features = torch.cat([
            mean_rgb, std_rgb, upper_mean, upper_std, lower_mean, lower_std
        ], dim=1)  # [B, 18]

        # Pad to desired dimension
        if color_features.size(1) < 64:
            padding = torch.zeros(batch_size, 64 - color_features.size(1),
                                device=x.device, dtype=x.dtype)
            color_features = torch.cat([color_features, padding], dim=1)

        return color_features[:, :64]

class ResidualBlock(nn.Module):
    """Residual block for better gradient flow"""
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()

        self.conv1 = nn.Conv2d(in_channels, out_channels, 3, stride, 1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, 3, 1, 1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, 1, stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ChannelAttention(nn.Module):
    """Channel attention module for feature refinement"""
    def __init__(self, in_channels, reduction=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)

        self.fc = nn.Sequential(
            nn.Conv2d(in_channels, in_channels // reduction, 1, bias=False),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels // reduction, in_channels, 1, bias=False)
        )

        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.fc(self.avg_pool(x))
        max_out = self.fc(self.max_pool(x))
        out = avg_out + max_out
        return x * self.sigmoid(out)

class AdvancedFeatureExtractor:
    """Advanced feature extraction with quality assessment"""
    def __init__(self, device='cuda'):
        self.device = device

        # Enhanced preprocessing pipeline
        self.transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((256, 128)),  # Higher resolution for better features
            transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.05),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                               std=[0.229, 0.224, 0.225])
        ])

        # Quality assessment thresholds
        self.min_size = (20, 40)
        self.max_blur_variance = 50
        self.min_brightness = 30
        self.max_brightness = 230

    def extract_roi_with_quality(self, bbox, frame):
        """Extract ROI with quality assessment"""
        try:
            x1, y1, x2, y2 = map(int, bbox)
            h, w = frame.shape[:2]

            # Clamp coordinates with margin
            margin = 5
            x1 = max(margin, min(x1, w - margin))
            y1 = max(margin, min(y1, h - margin))
            x2 = max(x1 + self.min_size[0], min(x2, w - margin))
            y2 = max(y1 + self.min_size[1], min(y2, h - margin))

            # Check minimum size
            if (x2 - x1) < self.min_size[0] or (y2 - y1) < self.min_size[1]:
                return None, 0.0

            # Extract ROI
            roi = frame[y1:y2, x1:x2]

            # Quality assessment
            quality_score = self._assess_roi_quality(roi)

            if quality_score < 0.3:  # Minimum quality threshold
                return None, quality_score

            # Convert to RGB and apply transforms
            roi_rgb = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
            tensor_roi = self.transform(roi_rgb)

            return tensor_roi, quality_score

        except Exception as e:
            return None, 0.0

    def _assess_roi_quality(self, roi):
        """Assess ROI quality for better feature extraction"""
        try:
            # Size score
            h, w = roi.shape[:2]
            area = h * w
            size_score = min(1.0, area / 10000.0)  # Normalize by expected area

            # Blur detection using Laplacian variance
            gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
            blur_var = cv2.Laplacian(gray, cv2.CV_64F).var()
            blur_score = min(1.0, blur_var / 200.0)

            # Brightness assessment
            brightness = np.mean(gray)
            if brightness < self.min_brightness or brightness > self.max_brightness:
                brightness_score = 0.5
            else:
                brightness_score = 1.0

            # Contrast assessment
            contrast = np.std(gray)
            contrast_score = min(1.0, contrast / 50.0)

            # Aspect ratio score (prefer human-like ratios)
            aspect_ratio = h / max(w, 1)
            if 1.5 <= aspect_ratio <= 3.5:
                aspect_score = 1.0
            else:
                aspect_score = 0.7

            # Combined quality score
            quality = (size_score * 0.2 +
                      blur_score * 0.3 +
                      brightness_score * 0.2 +
                      contrast_score * 0.2 +
                      aspect_score * 0.1)

            return quality

        except:
            return 0.1

    def extract_color_histogram(self, bbox, frame, bins=32):
        """Extract enhanced color histogram features"""
        try:
            x1, y1, x2, y2 = map(int, bbox)
            h, w = frame.shape[:2]

            # Clamp coordinates
            x1 = max(0, min(x1, w-1))
            y1 = max(0, min(y1, h-1))
            x2 = max(x1+1, min(x2, w))
            y2 = max(y1+1, min(y2, h))

            roi = frame[y1:y2, x1:x2]

            # Focus on upper body for jersey colors (top 60%)
            upper_h = int(roi.shape[0] * 0.6)
            upper_roi = roi[:upper_h, :]

            # Convert to multiple color spaces
            hsv_roi = cv2.cvtColor(upper_roi, cv2.COLOR_BGR2HSV)
            lab_roi = cv2.cvtColor(upper_roi, cv2.COLOR_BGR2LAB)

            # HSV histogram (better for clothing colors)
            hist_h = cv2.calcHist([hsv_roi], [0], None, [bins//2], [0, 180])
            hist_s = cv2.calcHist([hsv_roi], [1], None, [bins//4], [0, 256])
            hist_v = cv2.calcHist([hsv_roi], [2], None, [bins//4], [0, 256])

            # LAB histogram (perceptually uniform)
            hist_l = cv2.calcHist([lab_roi], [0], None, [bins//4], [0, 256])
            hist_a = cv2.calcHist([lab_roi], [1], None, [bins//4], [0, 256])
            hist_b = cv2.calcHist([lab_roi], [2], None, [bins//4], [0, 256])

            # Combine histograms
            combined_hist = np.concatenate([
                hist_h.flatten(),
                hist_s.flatten(),
                hist_v.flatten(),
                hist_l.flatten(),
                hist_a.flatten(),
                hist_b.flatten()
            ])

            # Normalize
            combined_hist = combined_hist / (combined_hist.sum() + 1e-6)

            return combined_hist

        except Exception as e:
            return np.zeros(bins * 6 // 4)  # Return zero vector on error

class EnhancedSportsTrack:
    """Enhanced track class with better feature management"""
    def __init__(self, track_id, detection, frame_idx):
        self.track_id = track_id
        self.bbox = detection['bbox'].copy()
        self.confidence = detection['confidence']
        self.class_id = detection['class_id']
        self.class_name = detection['class_name']
        self.birth_frame = frame_idx

        # Enhanced feature management
        self.reid_features = detection.get('reid_features', None)
        self.color_features = detection.get('color_features', None)
        self.quality_score = detection.get('quality_score', 0.5)

        # Feature history with quality weighting
        self.reid_history = deque(maxlen=20)  # Store more features
        self.color_history = deque(maxlen=15)
        self.quality_history = deque(maxlen=20)

        if self.reid_features is not None:
            self.reid_history.append(self.reid_features.copy())
            self.quality_history.append(self.quality_score)

        if self.color_features is not None:
            self.color_history.append(self.color_features.copy())

        # Enhanced state management
        self.age = 0
        self.hits = 1
        self.time_since_update = 0
        self.state = "tentative"

        # FIX: Initialize missing occlusion attributes
        self.was_occluded = False
        self.occlusion_count = 0

        # Adaptive thresholds based on quality
        base_max_disappeared = 30
        self.max_disappeared = int(base_max_disappeared * (1 + self.quality_score))

        # Motion modeling
        self.position_history = deque(maxlen=15)
        self.velocity_history = deque(maxlen=8)
        self.acceleration = np.array([0.0, 0.0])

        center = self._get_center()
        self.position_history.append(center)
        self.predicted_position = center.copy()

        # Enhanced confidence tracking
        self.confidence_history = deque([detection['confidence']], maxlen=15)
        self.avg_confidence = detection['confidence']
        self.peak_confidence = detection['confidence']

        # Track quality metrics
        self.track_stability = 0.0
        self.feature_consistency = 1.0

        # Team assignment
        self.team_id = None
        self.jersey_color = None
        self.dominant_color = None

    def _get_center(self):
        # FIX: Add bounds checking for bbox
        try:
            bbox = np.array(self.bbox, dtype=float)
            return np.array([(bbox[0] + bbox[2]) / 2,
                            (bbox[1] + bbox[3]) / 2])
        except:
            return np.array([0.0, 0.0])

    def update(self, detection, frame_idx):
        """Enhanced update with quality assessment"""
        self.age += 1
        self.hits += 1
        self.time_since_update = 0

        # Update position and motion
        try:
            new_center = np.array([(detection['bbox'][0] + detection['bbox'][2]) / 2,
                                 (detection['bbox'][1] + detection['bbox'][3]) / 2])
            self.position_history.append(new_center)
        except:
            pass

        # Update bbox and confidence with type checking
        try:
            self.bbox = np.array(detection['bbox'], dtype=float).copy()
            self.confidence = float(detection['confidence'])
            self.confidence_history.append(self.confidence)
        except:
            pass

        # Update quality score
        new_quality = detection.get('quality_score', 0.5)

        # Update peak confidence
        self.peak_confidence = max(self.peak_confidence, detection['confidence'])

        # Update average confidence with recency weighting
        if len(self.confidence_history) > 0:
            weights = np.exp(np.linspace(-1, 0, len(self.confidence_history)))
            weights /= weights.sum()
            self.avg_confidence = np.average(list(self.confidence_history), weights=weights)

        # Feature updates with quality weighting
        if 'reid_features' in detection and detection['reid_features'] is not None:
            self.reid_features = detection['reid_features']
            self.reid_history.append(detection['reid_features'].copy())
            self.quality_history.append(new_quality)

            # Update feature consistency
            if len(self.reid_history) >= 2:
                recent_features = np.array(list(self.reid_history)[-5:])  # Last 5 features
                similarities = []
                for i in range(len(recent_features)-1):
                    sim = np.dot(recent_features[i], recent_features[i+1])
                    similarities.append(sim)
                self.feature_consistency = np.mean(similarities) if similarities else 1.0

        if 'color_features' in detection and detection['color_features'] is not None:
            self.color_features = detection['color_features']
            self.color_history.append(detection['color_features'].copy())

        # Update track stability
        self._update_track_stability()

        # Adaptive state transitions
        confidence_threshold = 0.4 if self.feature_consistency > 0.7 else 0.5
        if self.hits >= 3 and self.avg_confidence > confidence_threshold:
            self.state = "confirmed"
        elif self.hits >= 8:  # Force confirmation for long tracks
            self.state = "confirmed"

    def _update_track_stability(self):
        """Update track stability metric"""
        if len(self.position_history) < 3:
            self.track_stability = 0.5
            return

        # Calculate position variance (lower is more stable)
        positions = np.array(list(self.position_history))
        if len(positions) >= 5:
            recent_positions = positions[-5:]
            pos_variance = np.mean(np.var(recent_positions, axis=0))
            # Normalize variance (assuming max reasonable variance is 10000)
            stability = max(0.0, 1.0 - pos_variance / 10000.0)
        else:
            stability = 0.5

        # Combine with confidence stability
        if len(self.confidence_history) >= 3:
            conf_variance = np.var(list(self.confidence_history))
            conf_stability = max(0.0, 1.0 - conf_variance)
        else:
            conf_stability = 0.5

        self.track_stability = 0.7 * stability + 0.3 * conf_stability

    def predict(self):
        """Enhanced prediction for sports movement"""
        if self.time_since_update == 0:
            return self.bbox.copy()

        # Calculate velocity from position history
        if len(self.position_history) >= 2:
            current_pos = self.position_history[-1]
            prev_pos = self.position_history[-2]
            velocity = current_pos - prev_pos

            # Add to velocity history
            self.velocity_history.append(velocity)

            # Calculate acceleration if we have velocity history
            if len(self.velocity_history) >= 2:
                current_vel = self.velocity_history[-1]
                prev_vel = self.velocity_history[-2]
                self.acceleration = 0.3 * (current_vel - prev_vel) + 0.7 * self.acceleration
        else:
            velocity = np.array([0.0, 0.0])

        # Predict next position with motion model
        if len(self.velocity_history) > 0:
            avg_velocity = np.mean(list(self.velocity_history), axis=0)
            # Scale prediction based on time since update
            prediction_scale = min(3.0, 1.0 + 0.5 * self.time_since_update)
            self.predicted_position = (self.position_history[-1] +
                                     avg_velocity * prediction_scale +
                                     0.5 * self.acceleration * prediction_scale**2)
        else:
            self.predicted_position = self.position_history[-1] if self.position_history else np.array([0.0, 0.0])

        # Update bbox based on predicted position
        try:
            current_width = self.bbox[2] - self.bbox[0]
            current_height = self.bbox[3] - self.bbox[1]

            # Gradual size increase for distant predictions
            size_scale = min(1.3, 1.0 + 0.05 * self.time_since_update)
            pred_width = current_width * size_scale
            pred_height = current_height * size_scale

            self.bbox = np.array([
                self.predicted_position[0] - pred_width/2,
                self.predicted_position[1] - pred_height/2,
                self.predicted_position[0] + pred_width/2,
                self.predicted_position[1] + pred_height/2
            ], dtype=float)
        except:
            pass

        return self.bbox.copy()

    def mark_missed(self):
        """Enhanced missed detection handling for sports"""
        self.time_since_update += 1

        # Check if likely occluded (based on track quality and history)
        if (self.get_track_score() > 0.6 and self.time_since_update < 15 and
            self.state == "confirmed"):
            self.was_occluded = True
            self.occlusion_count += 1

        # Dynamic deletion thresholds based on track quality and state
        if self.state == "confirmed":
            max_disappeared = self.max_disappeared
            if self.get_track_score() > 0.7:
                max_disappeared = 35  # Keep high-quality tracks longer
        else:
            max_disappeared = 8  # Quick deletion for tentative tracks

        if self.time_since_update > max_disappeared:
            self.state = "deleted"

    def get_weighted_reid_features(self):
        """Get quality-weighted average of ReID features"""
        if not self.reid_history or not self.quality_history:
            return self.reid_features

        try:
            features = np.array(list(self.reid_history))
            qualities = np.array(list(self.quality_history))

            # Apply recency weighting
            recency_weights = np.exp(np.linspace(-0.5, 0, len(features)))

            # Combine quality and recency weights
            combined_weights = qualities * recency_weights
            combined_weights = combined_weights / combined_weights.sum()

            weighted_features = np.average(features, axis=0, weights=combined_weights)
            return weighted_features

        except Exception as e:
            return self.reid_features if self.reid_features is not None else None

    def get_weighted_color_features(self):
        """Get weighted average of color features"""
        if not self.color_history:
            return self.color_features

        try:
            features = np.array(list(self.color_history))
            # Apply recency weighting
            weights = np.exp(np.linspace(-0.3, 0, len(features)))
            weights = weights / weights.sum()

            weighted_features = np.average(features, axis=0, weights=weights)
            return weighted_features

        except:
            return self.color_features

    def get_track_score(self):
        """Calculate comprehensive track score"""
        # Base score from confidence
        conf_score = self.avg_confidence

        # Stability bonus
        stability_bonus = self.track_stability * 0.2

        # Longevity bonus
        longevity_bonus = min(0.3, self.hits / 100.0)

        # Feature consistency bonus
        consistency_bonus = self.feature_consistency * 0.1

        # Quality bonus
        quality_bonus = self.quality_score * 0.1

        total_score = (conf_score + stability_bonus + longevity_bonus +
                      consistency_bonus + quality_bonus)

        return min(1.0, total_score)

class SportsYOLODetector:
    """YOLO detector optimized for sports/soccer scenarios"""
    def __init__(self, model_path='yolov8n.pt'):
        self.model = YOLO(model_path)
        self.class_names = self.model.names

        # Sports-specific parameters
        self.conf_threshold = 0.15  # Very low to catch distant players
        self.nms_threshold = 0.5
        self.target_classes = [0]  # Person class only

        # Field region detection (to be set during processing)
        self.field_mask = None
        self.field_bounds = None

    def set_field_region(self, frame):
        """Detect field region using color segmentation"""
        try:
            hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

            # Green field detection (adjust ranges as needed)
            lower_green = np.array([35, 40, 40])
            upper_green = np.array([85, 255, 255])

            mask = cv2.inRange(hsv, lower_green, upper_green)

            # Clean up mask
            kernel = np.ones((5,5), np.uint8)
            mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
            mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

            # Find largest contour (field)
            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            if contours:
                largest_contour = max(contours, key=cv2.contourArea)
                self.field_mask = np.zeros_like(mask)
                cv2.fillPoly(self.field_mask, [largest_contour], 255)

                # Get bounding rectangle
                x, y, w, h = cv2.boundingRect(largest_contour)
                self.field_bounds = (x, y, x+w, y+h)

        except Exception as e:
            print(f"Field detection error: {e}")
            self.field_mask = None

    def is_on_field(self, bbox):
        """Check if detection is on the field"""
        if self.field_mask is None:
            return True  # Accept all if no field mask

        # Check center point
        center_x = int((bbox[0] + bbox[2]) / 2)
        center_y = int((bbox[1] + bbox[3]) / 2)

        if (0 <= center_x < self.field_mask.shape[1] and
            0 <= center_y < self.field_mask.shape[0]):
            return self.field_mask[center_y, center_x] > 0

        return False

    def detect(self, frame):
        """Enhanced detection for sports scenarios"""
        # Set field region on first frame
        if self.field_mask is None:
            self.set_field_region(frame)

        # Run detection with low threshold
        results = self.model(
            frame,
            conf=self.conf_threshold,
            iou=self.nms_threshold,
            classes=self.target_classes,
            verbose=False
        )

        detections = []

        if results[0].boxes is not None:
            for box in results[0].boxes:
                bbox = box.xyxy[0].cpu().numpy()
                confidence = float(box.conf)
                class_id = int(box.cls)

                # Sports-specific filtering
                width = bbox[2] - bbox[0]
                height = bbox[3] - bbox[1]
                aspect_ratio = height / max(width, 1)

                # Size filtering (remove very small detections)
                if width < 15 or height < 30:
                    continue

                # Aspect ratio filtering for persons
                if aspect_ratio < 1.0 or aspect_ratio > 4.0:
                    continue

                # Field region filtering
                if not self.is_on_field(bbox):
                    continue

                # Area filtering (remove very large detections - likely false positives)
                area = width * height
                if area > frame.shape[0] * frame.shape[1] * 0.1:  # 10% of frame
                    continue

                detections.append({
                    'bbox': bbox,
                    'confidence': confidence,
                    'class_id': class_id,
                    'class_name': self.class_names[class_id],
                    'area': area,
                    'aspect_ratio': aspect_ratio
                })

        # Sort by confidence and return top detections
        detections = sorted(detections, key=lambda x: x['confidence'], reverse=True)
        return detections[:50]  # Limit to top 50 detections

class EnhancedSportsTracker:
    """Enhanced tracker with improved ReID feature generation"""
    def __init__(self, reid_model_path=None, max_tracks=25):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f"Using device: {self.device}")

        # Enhanced ReID model
        self.reid_model = EnhancedSportsReIDNet(feature_dim=768).eval().to(self.device)

        if reid_model_path and os.path.exists(reid_model_path):
            try:
                checkpoint = torch.load(reid_model_path, map_location=self.device)
                if 'state_dict' in checkpoint:
                    self.reid_model.load_state_dict(checkpoint['state_dict'])
                else:
                    self.reid_model.load_state_dict(checkpoint)
                print("Loaded custom ReID model")
            except Exception as e:
                print(f"ReID model load error: {e}")

        # Advanced feature extractor
        self.feature_extractor = AdvancedFeatureExtractor(device=self.device)

        self.tracks = []
        self.track_id_count = 1
        self.max_tracks = max_tracks

        # Enhanced association thresholds
        self.reid_threshold = 0.6  # Cosine similarity threshold
        self.color_threshold = 0.7  # Color similarity threshold
        self.iou_threshold = 0.1   # IoU threshold

        # Feature matching weights
        self.reid_weight = 0.6
        self.color_weight = 0.3
        self.iou_weight = 0.1

        self.frame_count = 0

        # Team clustering
        self.team_clusters = {}
        self.cluster_update_interval = 30  # Frames

        # Global track management
        self.global_track_registry = {}  # For handling re-appearances
        self.deleted_tracks_buffer = deque(maxlen=100)

        # Performance tracking
        self.processing_times = deque(maxlen=30)
        self.feature_extraction_times = deque(maxlen=30)

    def update(self, detections, frame=None):
        """Enhanced tracking update with improved ReID"""
        start_time = time.time()
        self.frame_count += 1

        try:
            # Filter detections
            valid_detections = self._filter_detections(detections, frame)

            # Extract features for all detections
            if frame is not None and len(valid_detections) > 0:
                feature_start = time.time()
                self._extract_enhanced_features(valid_detections, frame)
                feature_time = time.time() - feature_start
                self.feature_extraction_times.append(feature_time)

            # Predict track positions
            for track in self.tracks:
                track.predict()

            # Enhanced multi-stage association
            matched_pairs = self._enhanced_association(valid_detections)

            # Update matched tracks
            for track_idx, det_idx in matched_pairs:
                if track_idx < len(self.tracks) and det_idx < len(valid_detections):
                    self.tracks[track_idx].update(valid_detections[det_idx], self.frame_count)

            # Handle unmatched tracks
            matched_track_indices = [track_idx for track_idx, _ in matched_pairs]
            for i, track in enumerate(self.tracks):
                if i not in matched_track_indices:
                    track.mark_missed()

            # Create new tracks
            matched_det_indices = [det_idx for _, det_idx in matched_pairs]
            unmatched_detections = [
                (i, det) for i, det in enumerate(valid_detections)
                if i not in matched_det_indices
            ]

            self._create_enhanced_tracks(unmatched_detections)

            # Clean up tracks
            self._cleanup_tracks()

            # Update team clustering periodically
            if self.frame_count % self.cluster_update_interval == 0:
                self._update_team_clustering()

            # Track performance
            processing_time = time.time() - start_time
            self.processing_times.append(processing_time)

            return [t for t in self.tracks if t.state == "confirmed"]

        except Exception as e:
            print(f"Tracker error: {e}")
            return []

    def _extract_enhanced_features(self, detections, frame):
        """Extract enhanced features for all detections"""
        try:
            # Batch process ReID features
            valid_rois = []
            valid_indices = []
            quality_scores = []

            for idx, det in enumerate(detections):
                roi, quality = self.feature_extractor.extract_roi_with_quality(det['bbox'], frame)
                if roi is not None:
                    valid_rois.append(roi)
                    valid_indices.append(idx)
                    quality_scores.append(quality)

            # Extract ReID features in batches
            if valid_rois:
                batch_size = 8
                for i in range(0, len(valid_rois), batch_size):
                    batch_rois = valid_rois[i:i+batch_size]
                    batch_indices = valid_indices[i:i+batch_size]
                    batch_qualities = quality_scores[i:i+batch_size]

                    # Process batch
                    tensor_batch = torch.stack(batch_rois).to(self.device)

                    with torch.no_grad():
                        reid_features = self.reid_model(tensor_batch)
                        reid_features = reid_features.cpu().numpy()

                    # Assign features to detections
                    for j, det_idx in enumerate(batch_indices):
                        detections[det_idx]['reid_features'] = reid_features[j]
                        detections[det_idx]['quality_score'] = batch_qualities[j]

            # Extract color features for all detections
            for det in detections:
                if 'reid_features' in det:  # Only for valid detections
                    color_features = self.feature_extractor.extract_color_histogram(det['bbox'], frame)
                    det['color_features'] = color_features

        except Exception as e:
            print(f"Enhanced feature extraction error: {e}")

    def _enhanced_association(self, detections):
        """Enhanced multi-stage association with improved feature matching"""
        if not self.tracks or not detections:
            return []

        all_matches = []

        # Stage 1: High-quality ReID feature matching
        high_quality_tracks = []
        for i, track in enumerate(self.tracks):
            if (track.state == "confirmed" and
                track.get_track_score() > 0.7 and
                track.time_since_update <= 2 and
                track.get_weighted_reid_features() is not None):
                high_quality_tracks.append(i)

        high_quality_detections = []
        for i, det in enumerate(detections):
            if (det.get('quality_score', 0) > 0.6 and
                'reid_features' in det and det['reid_features'] is not None):
                high_quality_detections.append(i)

        if high_quality_tracks and high_quality_detections:
            stage1_matches = self._associate_by_enhanced_features(
                high_quality_tracks, high_quality_detections, detections
            )
            all_matches.extend(stage1_matches)

        # Stage 2: Medium quality feature matching
        matched_tracks = [t for t, _ in all_matches]
        matched_dets = [d for _, d in all_matches]

        medium_tracks = []
        for i, track in enumerate(self.tracks):
            if (i not in matched_tracks and
                track.state == "confirmed" and
                track.get_track_score() > 0.4 and
                track.time_since_update <= 5):
                medium_tracks.append(i)

        medium_detections = []
        for i, det in enumerate(detections):
            if (i not in matched_dets and
                det.get('quality_score', 0) > 0.3 and
                'reid_features' in det):
                medium_detections.append(i)

        if medium_tracks and medium_detections:
            stage2_matches = self._associate_by_enhanced_features(
                medium_tracks, medium_detections, detections, threshold=0.5
            )
            all_matches.extend(stage2_matches)

        # Stage 3: IoU-based matching for remaining
        matched_tracks = [t for t, _ in all_matches]
        matched_dets = [d for _, d in all_matches]

        remaining_tracks = [i for i in range(len(self.tracks)) if i not in matched_tracks]
        remaining_detections = [i for i in range(len(detections)) if i not in matched_dets]

        if remaining_tracks and remaining_detections:
            stage3_matches = self._associate_by_iou_enhanced(
                remaining_tracks, remaining_detections, detections
            )
            all_matches.extend(stage3_matches)

        return all_matches

    def _associate_by_enhanced_features(self, track_indices, det_indices, detections, threshold=None):
        """Enhanced feature-based association"""
        if not track_indices or not det_indices:
            return []

        if threshold is None:
            threshold = self.reid_threshold

        # Create comprehensive similarity matrix
        similarity_matrix = np.zeros((len(track_indices), len(det_indices)))

        for i, track_idx in enumerate(track_indices):
            track = self.tracks[track_idx]
            track_reid = track.get_weighted_reid_features()
            track_color = track.get_weighted_color_features()

            for j, det_idx in enumerate(det_indices):
                det = detections[det_idx]
                det_reid = det.get('reid_features')
                det_color = det.get('color_features')

                total_similarity = 0.0
                weight_sum = 0.0

                # ReID feature similarity
                if track_reid is not None and det_reid is not None:
                    try:
                        reid_sim = np.dot(track_reid, det_reid)
                        # Apply quality weighting
                        quality_weight = det.get('quality_score', 0.5) * track.get_track_score()
                        reid_sim *= quality_weight
                        total_similarity += reid_sim * self.reid_weight
                        weight_sum += self.reid_weight
                    except:
                        pass

                # Color feature similarity
                if track_color is not None and det_color is not None:
                    try:
                        # Use correlation coefficient for color histograms
                        color_corr = np.corrcoef(track_color.flatten(), det_color.flatten())[0, 1]
                        if not np.isnan(color_corr):
                            color_sim = max(0, color_corr)  # Only positive correlations
                            total_similarity += color_sim * self.color_weight
                            weight_sum += self.color_weight
                    except:
                        pass

                # IoU similarity (spatial consistency)
                try:
                    iou = self._calculate_iou(track.bbox, det['bbox'])
                    total_similarity += iou * self.iou_weight
                    weight_sum += self.iou_weight
                except:
                    pass

                # Motion consistency bonus
                if hasattr(track, 'predicted_position'):
                    try:
                        det_center = np.array([(det['bbox'][0] + det['bbox'][2]) / 2,
                                             (det['bbox'][1] + det['bbox'][3]) / 2])
                        distance = np.linalg.norm(track.predicted_position - det_center)
                        # Normalize distance by track size
                        track_size = max(track.bbox[2] - track.bbox[0], track.bbox[3] - track.bbox[1])
                        normalized_distance = distance / max(track_size, 1)
                        motion_bonus = max(0, 0.1 * (1 - min(normalized_distance / 2.0, 1.0)))
                        total_similarity += motion_bonus
                    except:
                        pass

                # Normalize by weight sum
                if weight_sum > 0:
                    similarity_matrix[i, j] = total_similarity / weight_sum

        # Hungarian assignment
        try:
            cost_matrix = 1 - similarity_matrix
            row_ind, col_ind = linear_sum_assignment(cost_matrix)

            matches = []
            for i, j in zip(row_ind, col_ind):
                if similarity_matrix[i, j] > threshold:
                    matches.append((track_indices[i], det_indices[j]))

            return matches

        except Exception as e:
            return []

    def _associate_by_iou_enhanced(self, track_indices, det_indices, detections):
        """Enhanced IoU-based association with motion prediction"""
        if not track_indices or not det_indices:
            return []

        # Create enhanced IoU matrix
        similarity_matrix = np.zeros((len(track_indices), len(det_indices)))

        for i, track_idx in enumerate(track_indices):
            track = self.tracks[track_idx]

            for j, det_idx in enumerate(det_indices):
                det = detections[det_idx]

                # Standard IoU
                iou = self._calculate_iou(track.bbox, det['bbox'])

                # Size consistency bonus
                try:
                    track_area = (track.bbox[2] - track.bbox[0]) * (track.bbox[3] - track.bbox[1])
                    det_area = (det['bbox'][2] - det['bbox'][0]) * (det['bbox'][3] - det['bbox'][1])

                    if track_area > 0 and det_area > 0:
                        area_ratio = min(track_area, det_area) / max(track_area, det_area)
                        size_bonus = 0.1 * area_ratio
                    else:
                        size_bonus = 0
                except:
                    size_bonus = 0

                # Confidence bonus
                conf_bonus = 0.05 * det['confidence']

                # Time penalty (tracks that haven't been updated)
                time_penalty = 0.05 * min(track.time_since_update, 5)

                total_similarity = iou + size_bonus + conf_bonus - time_penalty
                similarity_matrix[i, j] = max(0, total_similarity)

        # Hungarian assignment
        try:
            cost_matrix = 1 - similarity_matrix
            row_ind, col_ind = linear_sum_assignment(cost_matrix)

            matches = []
            for i, j in zip(row_ind, col_ind):
                if similarity_matrix[i, j] > self.iou_threshold:
                    matches.append((track_indices[i], det_indices[j]))

            return matches

        except:
            return []

    def _update_team_clustering(self):
        """Update team clustering based on color features"""
        try:
            # Collect color features from confirmed tracks
            color_features = []
            track_indices = []

            for i, track in enumerate(self.tracks):
                if (track.state == "confirmed" and
                    hasattr(track, 'color_features') and
                    track.color_features is not None):
                    color_features.append(track.color_features)
                    track_indices.append(i)

            if len(color_features) < 5:  # Not enough samples
                return

            # Convert to numpy array
            color_features = np.array(color_features)

            # Use DBSCAN for clustering
            dbscan = DBSCAN(eps=0.3, min_samples=3, metric='correlation')
            clusters = dbscan.fit_predict(color_features)

            # Assign team IDs based on clusters
            for idx, cluster_id in zip(track_indices, clusters):
                if cluster_id >= 0:  # -1 is noise in DBSCAN
                    self.tracks[idx].team_id = cluster_id
                    # Update jersey color based on cluster
                    if cluster_id == 0:
                        self.tracks[idx].jersey_color = "Team A"
                    elif cluster_id == 1:
                        self.tracks[idx].jersey_color = "Team B"
                    else:
                        self.tracks[idx].jersey_color = f"Team {cluster_id}"

            # Store clusters for reference
            self.team_clusters = {
                'last_update': self.frame_count,
                'n_clusters': len(set(clusters)) - (1 if -1 in clusters else 0)
            }

        except Exception as e:
            print(f"Team clustering error: {e}")

    def _create_enhanced_tracks(self, unmatched_detections):
        """Create new tracks with enhanced initialization"""
        for det_idx, det in unmatched_detections:
            if len(self.tracks) >= self.max_tracks:
                break

            # Quality-based track creation
            min_quality = 0.3
            min_confidence = 0.2

            if (det.get('quality_score', 0) > min_quality and
                det['confidence'] > min_confidence):

                new_track = EnhancedSportsTrack(
                    track_id=self.track_id_count,
                    detection=det,
                    frame_idx=self.frame_count
                )
                self.tracks.append(new_track)
                self.track_id_count += 1

    def _filter_detections(self, detections, frame):
        """Enhanced detection filtering"""
        if not detections:
            return []

        valid_detections = []
        frame_area = frame.shape[0] * frame.shape[1] if frame is not None else 640 * 480

        for det in detections:
            # Basic confidence filter
            if det['confidence'] < 0.15:
                continue

            # Enhanced bbox validation
            if not self._is_valid_bbox_enhanced(det['bbox'], frame.shape if frame is not None else None):
                continue

            # Size filtering
            width = det['bbox'][2] - det['bbox'][0]
            height = det['bbox'][3] - det['bbox'][1]
            area = width * height

            # Remove very small or very large detections
            if area < 400 or area > frame_area * 0.15:
                continue

            # Aspect ratio filtering
            aspect_ratio = height / max(width, 1)
            if aspect_ratio < 0.8 or aspect_ratio > 5.0:
                continue

            valid_detections.append(det)

        # Sort by confidence
        valid_detections.sort(key=lambda x: x['confidence'], reverse=True)

        # Limit number of detections
        return valid_detections[:self.max_tracks * 2]

    def _is_valid_bbox_enhanced(self, bbox, frame_shape):
        """Enhanced bounding box validation"""
        if frame_shape is not None:
            h, w = frame_shape[:2]

            # Check boundaries with margin
            margin = 10
            if (bbox[0] < -margin or bbox[1] < -margin or
                bbox[2] > w + margin or bbox[3] > h + margin):
                return False

            # Check minimum overlap with frame
            overlap_x1 = max(0, bbox[0])
            overlap_y1 = max(0, bbox[1])
            overlap_x2 = min(w, bbox[2])
            overlap_y2 = min(h, bbox[3])

            if overlap_x2 <= overlap_x1 or overlap_y2 <= overlap_y1:
                return False

            # Check minimum size
            if (bbox[2] - bbox[0]) < 10 or (bbox[3] - bbox[1]) < 20:
                return False

        # Check bbox consistency
        if bbox[0] >= bbox[2] or bbox[1] >= bbox[3]:
            return False

        return True

    def _calculate_iou(self, bbox1, bbox2):
        """Calculate Intersection over Union with error handling"""
        try:
            x1 = max(bbox1[0], bbox2[0])
            y1 = max(bbox1[1], bbox2[1])
            x2 = min(bbox1[2], bbox2[2])
            y2 = min(bbox1[3], bbox2[3])

            if x2 <= x1 or y2 <= y1:
                return 0.0

            intersection = (x2 - x1) * (y2 - y1)
            area1 = (bbox1[2] - bbox1[0]) * (bbox1[3] - bbox1[1])
            area2 = (bbox2[2] - bbox2[0]) * (bbox2[3] - bbox2[1])
            union = area1 + area2 - intersection

            if union <= 0:
                return 0.0

            return intersection / union

        except Exception as e:
            return 0.0

    def _cleanup_tracks(self):
        """Clean up tracks with enhanced criteria"""
        active_tracks = []

        for track in self.tracks:
            should_keep = True

            if hasattr(track, 'state') and track.state == "deleted":
                should_keep = False
            elif hasattr(track, 'time_since_update'):
                # Adaptive deletion based on track quality
                track_score = track.get_track_score() if hasattr(track, 'get_track_score') else 0.5

                if track.state == "confirmed":
                    max_disappeared = int(30 * (1 + track_score))
                else:
                    max_disappeared = 8

                if track.time_since_update > max_disappeared:
                    should_keep = False

            if should_keep:
                active_tracks.append(track)

        self.tracks = active_tracks

    def get_performance_stats(self):
        """Get tracker performance statistics"""
        stats = {}

        if self.processing_times:
            avg_fps = 1.0 / np.mean(self.processing_times)
            stats['avg_fps'] = avg_fps
            stats['avg_processing_time'] = np.mean(self.processing_times)

        if self.feature_extraction_times:
            stats['avg_feature_time'] = np.mean(self.feature_extraction_times)
            stats['feature_time_ratio'] = (np.mean(self.feature_extraction_times) /
                                          max(np.mean(self.processing_times), 0.001))

        stats['active_tracks'] = len([t for t in self.tracks if t.state == "confirmed"])
        stats['total_tracks'] = len(self.tracks)
        stats['tracks_created'] = self.track_id_count - 1

        return stats

class SportsVideoProcessor:
    """Video processor optimized for sports tracking"""
    def __init__(self, model_path=None, max_frames=1000):
        print("Initializing Sports Video Processor...")
        self.tracker = EnhancedSportsTracker(reid_model_path=model_path)
        self.detector = SportsYOLODetector()
        self.max_frames = max_frames
        self.frame_skip = 1
        self.processed_frames = 0

        # Visualization settings
        self.colors = [(np.random.randint(50, 255),
                       np.random.randint(50, 255),
                       np.random.randint(50, 255)) for _ in range(100)]

        # Team colors (can be customized)
        self.team_colors = {
            "Team A": (255, 0, 0),    # Red
            "Team B": (0, 0, 255),    # Blue
            "Referee": (0, 255, 0),   # Green
            None: (255, 255, 255)     # White for unknown
        }

        # Performance tracking
        self.processing_times = deque(maxlen=100)
        self.start_time = None

    def process_frame(self, frame, frame_idx):
        """Process a single frame"""
        try:
            frame_start = time.time()

            # Detect objects
            detections = self.detector.detect(frame)

            # Update tracker
            tracks = self.tracker.update(detections, frame)

            # Visualize results
            vis_frame = self._visualize(frame, tracks, detections, frame_idx)

            # Performance tracking
            processing_time = time.time() - frame_start
            self.processing_times.append(processing_time)

            return vis_frame, tracks, True

        except Exception as e:
            print(f"Frame {frame_idx} processing error: {e}")
            return frame.copy(), [], False

    def _visualize(self, frame, tracks, detections, frame_idx):
        """Enhanced visualization for sports tracking - FIX: Added proper type checking"""
        vis_frame = frame.copy()

        # Draw field boundaries if available
        if self.detector.field_bounds is not None:
            try:
                x1, y1, x2, y2 = map(int, self.detector.field_bounds)
                cv2.rectangle(vis_frame, (x1, y1), (x2, y2), (0, 255, 0), 1)
            except:
                pass

        # Draw detections (transparent)
        for det in detections:
            try:
                bbox = det['bbox']
                if bbox is not None and len(bbox) >= 4:
                    x1, y1, x2, y2 = map(int, bbox[:4])
                    cv2.rectangle(vis_frame, (x1, y1), (x2, y2), (200, 200, 200), 1)
            except:
                continue

        # Draw tracks with team information
        for track in tracks:
            try:
                # Determine color based on team
                if hasattr(track, 'jersey_color') and track.jersey_color in self.team_colors:
                    color = self.team_colors[track.jersey_color]
                elif hasattr(track, 'team_id'):
                    color = self.team_colors.get(f"Team {track.team_id}", (255, 255, 255))
                else:
                    color = self.colors[track.track_id % len(self.colors)]

                # FIX: Ensure bbox is valid before drawing
                bbox = track.bbox
                if bbox is None or len(bbox) < 4:
                    continue

                # Convert to integers with bounds checking
                try:
                    bbox_int = [int(float(x)) for x in bbox[:4]]
                    x1, y1, x2, y2 = bbox_int

                    # Sanity check coordinates
                    if x1 >= x2 or y1 >= y2:
                        continue

                except (ValueError, TypeError, OverflowError):
                    continue

                # Draw bounding box
                thickness = 3 if track.state == "confirmed" else 1
                cv2.rectangle(vis_frame, (x1, y1), (x2, y2), color, thickness)

                # Draw team/jersey information if available
                team_text = ""
                if hasattr(track, 'jersey_color'):
                    team_text = track.jersey_color
                elif hasattr(track, 'team_id') and track.team_id is not None:
                    team_text = f"Team {track.team_id}"

                # Create label
                label_parts = [f"ID:{track.track_id}"]
                if team_text:
                    label_parts.append(team_text)
                if hasattr(track, 'time_since_update') and track.time_since_update > 0:
                    label_parts.append(f"({track.time_since_update})")

                label = " ".join(label_parts)

                # Draw label background
                try:
                    (text_w, text_h), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
                    cv2.rectangle(vis_frame, (x1, y1 - text_h - 5), (x1 + text_w + 5, y1), color, -1)

                    # Draw label text
                    cv2.putText(vis_frame, label, (x1 + 2, y1 - 5),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
                except:
                    pass

                # Draw trajectory
                if hasattr(track, 'position_history') and len(track.position_history) > 1:
                    try:
                        points = np.array(track.position_history, dtype=np.int32)
                        if len(points) > 1:
                            cv2.polylines(vis_frame, [points], False, color, 1)
                    except:
                        pass

            except Exception as e:
                # Skip this track if there's any error in visualization
                continue

        # Draw status information
        status_info = [
            f"Frame: {frame_idx}",
            f"Detections: {len(detections)}",
            f"Tracks: {len(tracks)}",
            f"Teams: {self.tracker.team_clusters.get('n_clusters', 0)}",
            f"FPS: {1.0/np.mean(self.processing_times):.1f}" if self.processing_times else "FPS: --"
        ]

        for i, text in enumerate(status_info):
            try:
                cv2.putText(vis_frame, text, (10, 20 + i*20),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
            except:
                pass

        return vis_frame

    def process_video(self, input_path, output_path):
        """Process video file"""
        print(f"\nProcessing video: {input_path}")
        print(f"Output will be saved to: {output_path}")

        cap = cv2.VideoCapture(input_path)
        if not cap.isOpened():
            print(f"Error opening video: {input_path}")
            return False

        # Get video properties
        fps = cap.get(cv2.CAP_PROP_FPS)
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        # Setup output video
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

        if not out.isOpened():
            print(f"Error creating output video: {output_path}")
            cap.release()
            return False

        self.start_time = time.time()
        frame_idx = 0

        while cap.isOpened() and frame_idx < self.max_frames:
            ret, frame = cap.read()
            if not ret:
                break

            if frame_idx % self.frame_skip == 0:
                # Process frame
                vis_frame, _, success = self.process_frame(frame, frame_idx)

                if success:
                    out.write(vis_frame)
                    self.processed_frames += 1

                # Print progress
                if frame_idx % 50 == 0:
                    elapsed = time.time() - self.start_time
                    fps = (frame_idx + 1) / elapsed if elapsed > 0 else 0
                    remaining = (total_frames - frame_idx) / max(fps, 0.1) / 60
                    print(f"Processed {frame_idx}/{total_frames} frames | "
                          f"FPS: {fps:.1f} | "
                          f"ETA: {remaining:.1f} min | "
                          f"Tracks: {len(self.tracker.tracks)}")

            frame_idx += 1

        # Release resources
        cap.release()
        out.release()

        # Print summary
        self._print_summary()

        return True

    def _print_summary(self):
        """Print processing summary"""
        if self.start_time is None:
            return

        duration = time.time() - self.start_time
        avg_fps = self.processed_frames / duration if duration > 0 else 0

        print("\n" + "="*50)
        print("Processing Summary")
        print("="*50)
        print(f"Total frames processed: {self.processed_frames}")
        print(f"Total time: {duration:.1f} seconds")
        print(f"Average FPS: {avg_fps:.1f}")
        print(f"Max tracks at once: {self.tracker.max_tracks}")
        print(f"Total tracks created: {self.tracker.track_id_count - 1}")

        if hasattr(self.tracker, 'team_clusters'):
            print(f"Teams detected: {self.tracker.team_clusters.get('n_clusters', 0)}")

        print("="*50)

if __name__ == "__main__":
    # Configuration - SET YOUR PATHS HERE
    input_video = "/content/data/video.mp4"  # Your video path
    output_video = "/content/output_tracked.mp4"  # Output path
    reid_model_path = None  # Optional path to custom ReID model

    # Initialize processor
    processor = SportsVideoProcessor(
        model_path=reid_model_path,
        max_frames=1000  # Set to None to process entire video
    )

    # Verify input file exists
    if not os.path.exists(input_video):
        print(f"\nERROR: Input video not found at {input_video}")
        print("Please verify the path exists and try again.")
        exit(1)

    # Process video
    success = processor.process_video(input_video, output_video)

    if success:
        print(f"\n✅ Processing completed successfully!")
        print(f"Output saved to: {output_video}")
    else:
        print("\n❌ Processing failed")

Initializing Sports Video Processor...
Using device: cpu

Processing video: /content/data/video.mp4
Output will be saved to: /content/output_tracked.mp4
Processed 0/449 frames | FPS: 0.9 | ETA: 7.9 min | Tracks: 20
Processed 50/449 frames | FPS: 1.6 | ETA: 4.1 min | Tracks: 25
Processed 100/449 frames | FPS: 1.3 | ETA: 4.5 min | Tracks: 24
Processed 150/449 frames | FPS: 1.2 | ETA: 4.2 min | Tracks: 25
Processed 200/449 frames | FPS: 1.1 | ETA: 3.6 min | Tracks: 22
Processed 250/449 frames | FPS: 1.1 | ETA: 3.0 min | Tracks: 22
Processed 300/449 frames | FPS: 1.1 | ETA: 2.2 min | Tracks: 18
Processed 350/449 frames | FPS: 1.1 | ETA: 1.5 min | Tracks: 22
Processed 400/449 frames | FPS: 1.1 | ETA: 0.7 min | Tracks: 21

Processing Summary
Total frames processed: 449
Total time: 397.4 seconds
Average FPS: 1.1
Max tracks at once: 25
Total tracks created: 120
Teams detected: 1

✅ Processing completed successfully!
Output saved to: /content/output_tracked.mp4


In [5]:
%pip install ultralytics